# Машинное обучение 2020, часть 2, семинар 7. Векторные представления слов и механизм внимания

In [ ]:
import os
import sys
import torch
import numpy as np
import torch.optim as optim
import torch.nn.functional as F

from itertools import chain
from collections import Counter
from tqdm.notebook import tqdm

from torch import nn
from torch.optim import SGD, Adagrad, Adam

Сегодня мы будем с вами решать задачу part-of-speech tagging'а.

## Датасет

Далее будем работать с размеченным корпусом [SynTagRus](https://github.com/UniversalDependencies/UD_Russian-SynTagRus). 

Более подробнее о нём можно прочитать [тут](https://ruscorpora.ru/new/instruction-syntax.html), а также в [данной публикации](http://iitp.ru/upload/publications/8090/2019_SynTagRus_segodnya.pdf).

In [ ]:
!git clone https://github.com/UniversalDependencies/UD_Russian-SynTagRus.git

Cloning into 'UD_Russian-SynTagRus'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 321 (delta 3), reused 4 (delta 1), pack-reused 308
Receiving objects: 100% (321/321), 180.75 MiB | 26.78 MiB/s, done.
Resolving deltas: 100% (196/196), done.


In [ ]:
DATA_PATH = 'UD_Russian-SynTagRus'

In [ ]:
with open(os.path.join(DATA_PATH, 'ru_syntagrus-ud-train.conllu')) as file:
    X_train = file.read()

print(X_train[:1000])

# sent_id = 2003Anketa.xml_1
# text = Анкета.
1	Анкета	анкета	NOUN	_	Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing	0	root	0:root	SpaceAfter=No
2	.	.	PUNCT	_	_	1	punct	1:punct	_

# sent_id = 2003Anketa.xml_2
# text = Начальник областного управления связи Семен Еремеевич был человек простой, приходил на работу всегда вовремя, здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом "Муха".
1	Начальник	начальник	NOUN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	8	nsubj	8:nsubj	_
2	областного	областной	ADJ	_	Case=Gen|Degree=Pos|Gender=Neut|Number=Sing	3	amod	3:amod	_
3	управления	управление	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	1	nmod	1:nmod	_
4	связи	связь	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	3	nmod	3:nmod	_
5	Семен	Семен	PROPN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	1	appos	1:appos	_
6	Еремеевич	Еремеевич	PROPN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	5	flat:name	5:flat:name	_
7	был	быть	AUX	_	Aspect=Imp|Gend

Структура странноватая... Значит надо написать парсер, преобразующий в удобоваримый вид.

Более того, нам понадобятся только нормальная форма слова, а также его часть речи (в качестве ответа):

In [ ]:
def load_data(path):
    with open(path) as file:
        data = file.read()

    # split into sentences
    data = data.split('\n\n')
    # split into tokens
    data = [sent.split('\n')[3:] for sent in data]
    # get lemma and pos-tag
    data = [[token.split('\t')[2:4] for token in sent] for sent in data]

    return data

In [ ]:
data_train = load_data(os.path.join(DATA_PATH, 'ru_syntagrus-ud-train.conllu'))
data_dev = load_data(os.path.join(DATA_PATH, 'ru_syntagrus-ud-dev.conllu'))
data_test = load_data(os.path.join(DATA_PATH, 'ru_syntagrus-ud-test.conllu'))

In [ ]:
data_train[3]

[['стиль', 'NOUN'],
 ['работа', 'NOUN'],
 ['Семен', 'PROPN'],
 ['Еремеевич', 'PROPN'],
 ['заключаться', 'VERB'],
 ['в', 'ADP'],
 ['то', 'PRON'],
 [',', 'PUNCT'],
 ['чтобы', 'SCONJ'],
 ['принимать', 'VERB'],
 ['весь', 'DET'],
 ['желать', 'VERB'],
 ['и', 'CCONJ'],
 ['лично', 'ADV'],
 ['вникать', 'VERB'],
 ['в', 'ADP'],
 ['дело', 'NOUN'],
 ['.', 'PUNCT']]

Обычно для задач, связанных с обработкой текста, нужна некоторая предобработка данных.

Хорошо, что хотя бы привели слово к нормальной форме за нас!

Для начала посчитаем общее количество токенов в нашем датасете:

In [ ]:
tokens, tags = zip(*chain(*(data_train + data_dev + data_test)))
print(f'Всего {len(tokens)} токенов в корпусе')

Всего 1045852 токенов в корпусе


Довольно логично, что если слово встречается в датасете очень редко, то особо ничего нам про него выучить не получится. Давайте будем обрабатывать только те слова, которые встречаются как минимум 3 раза:

In [ ]:
min_freq = 3
counts = Counter(tokens)
tokens = [token for token in tokens if counts[token] >= min_freq]

print(f'Всего {len(tokens)} частотных токенов в корпусе')

Всего 1013962 частотных токенов в корпусе


Слов получилось довольно много, всё будет работать довольно долго... Хотелось бы уменьшить их количество. При этом просто убирать самые нечастые как-то неправильно. Давайте просто посэмплируем пропорционально частоте встречаемости слов:

In [ ]:
counts = Counter(tokens)
total_tokens = sum(counts.values())

subsample_t = 10**(-3)
subsample_p = {token: max(0, 1 - np.sqrt(subsample_t / (counts[token] / total_tokens))) for token in tokens}

tokens = [token for token in tokens if np.random.binomial(1, 1 - subsample_p[token])]

print(f'Всего {len(tokens)} токенов осталось в корпусе после subsampling')

Всего 702186 токенов осталось в корпусе после subsampling


А теперь уже можно и словарь формировать:

In [ ]:
counts = Counter(tokens)
vocabulary = ['<UNK>'] + sorted(counts.keys())

print(f'Всего {len(vocabulary)} токенов в словаре')

Всего 19467 токенов в словаре


In [ ]:
token2index = {token: index for index, token in enumerate(vocabulary)}
tag2index = {tag: index for index, tag in enumerate(set(tags))}

In [ ]:
tag2index

{'ADJ': 4,
 'ADP': 10,
 'ADV': 8,
 'AUX': 6,
 'CCONJ': 16,
 'DET': 2,
 'INTJ': 12,
 'NOUN': 17,
 'NUM': 15,
 'PART': 9,
 'PRON': 3,
 'PROPN': 14,
 'PUNCT': 11,
 'SCONJ': 7,
 'SYM': 0,
 'VERB': 1,
 'X': 13,
 '_': 5}

## Простые эмбединги

Самый простой способ представления слов - в виде one-hot векторов. Но это как-то совсем неудобно. Будем пользоваться векторными представлениями, но учить их будем только для нашей задачи.


А самая простая модель для моделирования последовательностей - это RNN. Реализуем простейшую рекуррентную нейросеть для решения нашей задачи:

![](https://www.mdpi.com/algorithms/algorithms-10-00037/article_deploy/html/images/algorithms-10-00037-g010.png)

In [ ]:
class RNNTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(RNNTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.RNN(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Обучать будем её на минимизацию кросс-энтропии (как это часто делается в задачах мультиклассовой классификации):

In [ ]:
rnn_model = RNNTagger(32, 32, len(vocabulary), len(tag2index))

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.1)

Конечно же данные тоже надо подготовить соответствующим образом:

In [ ]:
def prepare_sent(sent):
    new_sent = []
    for token, tag in sent:
        if token not in token2index:
            token = '<UNK>'
        new_sent.append((token2index[token], tag2index[tag]))
        
    if not new_sent:
        return None, None

    token_indecies, tag_indecies = zip(*new_sent)
    token_indecies = torch.tensor(token_indecies, dtype=torch.long)
    tag_indecies = torch.tensor(tag_indecies, dtype=torch.long)
    
    return token_indecies, tag_indecies

In [ ]:
tokens, tags = prepare_sent(data_train[3])

И, наконец, тренировка модели:

In [ ]:
from tqdm.notebook import tqdm

def train_rnn(model, loss, optimizer, data_train, data_val, num_epoch=3):
    for epoch in range(num_epoch):
        print('Epoch: ', epoch)
        epoch_loss = 0
        print('Train:')
        for sent in tqdm(data_train):
            model.zero_grad()

            tokens, tags = prepare_sent(sent)
            if tokens is None or tags is None:
                continue

            tag_scores = model(tokens)

            sent_loss = loss(tag_scores, tags)
            sent_loss.backward()
            optimizer.step()

            epoch_loss += float(sent_loss.data)
            
        print('Train loss: ', epoch_loss)

        epoch_loss = 0
        print('Validation:')
        for sent in tqdm(data_val):
            tokens, tags = prepare_sent(sent)
            if tokens is None or tags is None:
                continue

            tag_scores = model(tokens)
            sent_loss = loss(tag_scores, tags)

            epoch_loss += float(sent_loss.data)
        print('Validation loss: ', epoch_loss)

На google colab обучение занимает примерно 15 минут:

In [ ]:
train_rnn(rnn_model, loss, optimizer, data_train, data_dev)

Epoch:  0
Train loss:  29103.641500720747
Validation loss:  3494.56474433087
Epoch:  1
Train loss:  16045.492292821671
Validation loss:  2685.81934238561
Epoch:  2
Train loss:  11485.091805010634
Validation loss:  2301.96986571914


In [ ]:
with torch.no_grad():
    token_indecies, tag_indecies = prepare_sent(data_test[0])
    tag_scores = rnn_model(token_indecies)
    
    print(tag_scores)

tensor([[-1.8047e+01, -9.4578e+00, -1.1152e+01, -2.0541e+01, -9.6695e+00,
         -1.3003e+01, -1.7247e+01, -1.9471e+01, -1.3263e+01, -5.9950e+00,
         -1.6521e+01, -1.2367e+01, -1.8922e+01, -1.9057e+01, -2.6601e-03,
         -1.5043e+01, -1.4462e+01, -1.4036e+01],
        [-2.3888e+01, -6.9977e+00, -9.4893e+00, -2.0987e+01, -1.8710e+01,
         -1.9091e+01, -3.1940e+01, -2.0384e+01, -9.5317e+00, -8.0159e+00,
         -2.1700e+01, -1.3516e+01, -1.8023e+01, -2.4818e+01, -4.6226e+00,
         -1.1287e-02, -1.6475e+01, -1.2759e+01],
        [-2.4288e+01, -1.0278e+01, -1.2773e+01, -2.8452e+01, -2.6279e+01,
         -2.7443e+01, -3.2156e+01, -3.0136e+01, -2.0112e+01, -9.4282e+00,
         -2.0866e+01, -2.1607e+01, -2.7213e+01, -1.6436e+01, -1.0047e+01,
         -1.6116e-04, -3.0836e+01, -1.6217e+01],
        [-2.1055e+01, -7.1007e+00, -2.8537e+00, -1.6153e+01, -1.5407e+01,
         -1.3916e+01, -2.1867e+01, -2.0242e+01, -1.1103e+01, -2.9349e+00,
         -2.4067e+01, -1.4953e+01, -2.0

Интересно, насколько адекватные у нас получились эмбединги? Ведь мы учили их только лишь для нашей задачи, а не из каких-то общих соображений.

**Задача 1 (1 балл)**. Модифицируйте модель так, чтобы она могла выдавать эмбединги для слов. Изучите эти эмбединги. Для этого реализуйте и воспользуйтесь для исследования следующими функциями:
* поиск ближайших слов к данному
* ''близость'' двух слов

In [ ]:
class RNN_Emb(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(RNN_Emb, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.RNN(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

    def embeddings(self):
      return self.word_embeddings.weight.data.numpy()

In [ ]:
rnn_emb_model = RNN_Emb(32, 32, len(vocabulary), len(tag2index))

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(rnn_emb_model.parameters(), lr=0.1)

In [ ]:
train_rnn(rnn_emb_model, loss, optimizer, data_train, data_dev, num_epoch=10)

Epoch:  0
Train:



Train loss:  30196.021996484982
Validation:



Validation loss:  3635.947283528454
Epoch:  1
Train:



Train loss:  16693.67444865444
Validation:



Validation loss:  2749.0854178874724
Epoch:  2
Train:


...


Validation loss:  1727.5002909164637
Epoch:  8
Train:



Train loss:  6242.839748630058
Validation:



Validation loss:  1684.5092144118437
Epoch:  9
Train:



Train loss:  6016.9056007768
Validation:



Validation loss:  1661.4386270422046


In [ ]:
embeddings = rnn_emb_model.embeddings()
embeddings.shape

(19467, 32)

In [ ]:
def get_most_similar_words(word, embeddings, topn=10):
  ind = token2index[word]
  emb = embeddings[ind]

  prod = emb @ embeddings.T
  norm = np.linalg.norm(embeddings, axis=1)

  dist = prod / norm
  dist = dist / np.linalg.norm(emb)
  dist = np.abs(dist)

  top_sim_words = np.argsort(dist)[1:(topn+1)]
  top_sim_words = [vocabulary[ind] for ind in top_sim_words]

  return top_sim_words


In [ ]:
get_most_similar_words('женщина', embeddings)

['принять',
 'Уокер',
 'минимально',
 'рукотворный',
 'переставать',
 'Питер',
 'ватник',
 'рукоятка',
 'ваза',
 '1990-й']

In [ ]:
get_most_similar_words('снег', embeddings)

['аппарат',
 'спекулятивный',
 'маловероятный',
 'реанимация',
 'наводнение',
 'династия',
 'рассекать',
 'тумбочка',
 'негосударственный',
 'про']

In [ ]:
def words_cosine_similarity(word_a, word_b, embeddings):
  ind_a = token2index[word_a]
  ind_b = token2index[word_b]

  emb_a = embeddings[ind_a]
  emb_b = embeddings[ind_b]

  cosine = np.sum(emb_a * emb_b)
  cosine /= np.linalg.norm(emb_a)
  cosine /= np.linalg.norm(emb_b)

  return cosine


In [ ]:
print(f"{words_cosine_similarity('папа', 'папа', embeddings):.4f}")
print(f"{words_cosine_similarity('папа', 'мама', embeddings):.4f}")
print(f"{words_cosine_similarity('Алжир', 'снег', embeddings):.4f}")
print(f"{words_cosine_similarity('женщина', 'мужчина', embeddings):.4f}")
print(f"{words_cosine_similarity('добро', 'зло', embeddings):.4f}")

1.0000
0.0518
0.0607
-0.0274
0.1241


Самый адекватный результат - это то, что косинус между эмбедингами для одного и того же слова = 1.

## Word2vec

Давайте лучше обучим какие-то более содержательные эмбединги. Например, обучим подобие word2vec с помощью модели CBOW:

![](https://www.researchgate.net/profile/Daniel_Braun6/publication/326588219/figure/fig1/AS:652185784295425@1532504616288/Continuous-Bag-of-words-CBOW-CB-and-Skip-gram-SG-training-model-illustrations.png)

In [ ]:
class Word2VecModel(nn.Module):
    def __init__(self, embedding_dim, vocab_size):
        super(Word2VecModel, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(2 * embedding_dim, vocab_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        pre_embeds = embeds[:-2]
        post_embeds = embeds[2:]
        combined_embeds = torch.cat((pre_embeds, post_embeds), dim=1)
        words_scores = F.log_softmax(self.linear(combined_embeds), dim=1)
        return words_scores
    
    def embeddings(self):
      return self.word_embeddings.weight.data.numpy()

In [ ]:
def prepare_data(sent):
    new_sent = []
    for token, tag in sent:
        if token not in token2index:
            token = '<UNK>'
        new_sent.append(token2index[token])
        
    if not new_sent:
        return None

    token_indecies = torch.tensor(new_sent, dtype=torch.long)
    return token_indecies

new_x = prepare_data(data_train[3])
print(new_x)

tensor([16943, 14724,  2478,  1417,  6817,  4104, 17494,     8, 18982, 14022,
         4290,  6529,  7301,  8993,  4520,  4104,  5901,    10])


In [ ]:
def train_word2vec(model, loss, optimizer, data_train, data_val, num_epoch=3):
    for epoch in range(num_epoch):
        print('Epoch: ', epoch)
        epoch_loss = 0
        for sent in tqdm(data_train):
            model.zero_grad()

            tokens = prepare_data(sent)
            if tokens is None or len(tokens) < 3:
                continue

            words_scores = model(tokens)

            sent_loss = loss(words_scores, tokens[1:-1])
            sent_loss.backward()
            optimizer.step()

            epoch_loss += float(sent_loss.data)
            
        print('Train loss: ', epoch_loss)

        epoch_loss = 0
        for sent in data_val:
            tokens = prepare_data(sent)
            if tokens is None:
                continue

            words_scores = model(tokens)
            sent_loss = loss(words_scores, tokens[1:-1])

            epoch_loss += float(sent_loss.data)
        print('Validation loss: ', epoch_loss)

In [ ]:
model_word2vec = Word2VecModel(32, len(vocabulary))
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_word2vec.parameters(), lr=0.1)

train_word2vec(model_word2vec, loss, optimizer, data_train, data_dev)

NameError: ignored

**Задача 2 (1 балл)**. Два слова хорошо - а четыре лучше! Модицифируйте модель и её обучение так, чтобы можно было настраивать контекст какого размера использовать. А именно, сколько эмбедингов брать до самого и после самого слова для обучения модели его восстановления.

In [ ]:
class Word2VecModelContext(nn.Module):
    def __init__(self, embedding_dim, vocab_size, window_halfsize=2):
        super(Word2VecModelContext, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(2 * embedding_dim * window_halfsize, vocab_size)
        self.window_halfsize = window_halfsize

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        # pre_embeds = embeds[:-2]
        # post_embeds = embeds[2:]
        context = []
        for i in range(self.window_halfsize):
          context.append(embeds[self.window_halfsize-i-1:-self.window_halfsize-i-1])
          context.append(embeds[self.window_halfsize-i+1:-self.window_halfsize-i+1])
          # print(self.window_halfsize-i-1, -self.window_halfsize-i-1)
          # print(self.window_halfsize-i+1, -self.window_halfsize-i+1)
        combined_embeds = torch.cat(context, dim=1)
        words_scores = F.log_softmax(self.linear(combined_embeds), dim=1)
        return words_scores
    
    def embeddings(self):
      return self.word_embeddings.weight.data.numpy()

In [ ]:
def train_word2vec_context(model, loss, optimizer, data_train, 
                           data_val, num_epoch=3, window_halfsize=2):
    for epoch in range(num_epoch):
        print('Epoch: ', epoch)
        epoch_loss = 0
        print('Train: ')
        for sent in tqdm(data_train):
            model.zero_grad()

            tokens = prepare_data(sent)
            if tokens is None or len(tokens) < 2*window_halfsize + 1:
                continue

            words_scores = model(tokens)

            sent_loss = loss(words_scores, tokens[window_halfsize:-window_halfsize])
            sent_loss.backward()
            optimizer.step()

            epoch_loss += float(sent_loss.data)
            
        print('Train loss: ', epoch_loss)

        epoch_loss = 0
        print('Validation: ')
        for sent in tqdm(data_val):
            tokens = prepare_data(sent)
            if tokens is None or len(tokens) < 2*window_halfsize + 1:
                continue

            words_scores = model(tokens)
            sent_loss = loss(words_scores, tokens[window_halfsize:-window_halfsize])

            epoch_loss += float(sent_loss.data)
        print('Validation loss: ', epoch_loss)

In [ ]:
window_halfsize = 2
num_epoch = 5

In [ ]:
model_word2vec = Word2VecModelContext(32, len(vocabulary), 2)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_word2vec.parameters(), lr=0.1)

train_word2vec_context(model_word2vec, loss, optimizer, data_train, 
                       data_dev, num_epoch=num_epoch, 
                       window_halfsize=window_halfsize)

Epoch:  0
Train: 



Train loss:  110785.57147297665
Validation: 



Validation loss:  9924.406994585239
Epoch:  1
Train: 



Train loss:  43247.32772452758
Validation: 



Validation loss:  7592.412031925891
Epoch:  2
Train: 


...


Validation loss:  6553.860938192884
Epoch:  3
Train: 



Train loss:  17030.28893442086
Validation: 



Validation loss:  5967.088049851648
Epoch:  4
Train: 



Train loss:  11869.748992119596
Validation: 



Validation loss:  5591.691144585718


In [ ]:
w2v_embeddings = model_word2vec.embeddings()

print(f"{words_cosine_similarity('папа', 'папа', w2v_embeddings):.4f}")
print(f"{words_cosine_similarity('кот', 'собака', w2v_embeddings):.4f}")
print(f"{words_cosine_similarity('папа', 'мама', w2v_embeddings):.4f}")
print(f"{words_cosine_similarity('Алжир', 'снег', w2v_embeddings):.4f}")
print(f"{words_cosine_similarity('женщина', 'мужчина', w2v_embeddings):.4f}")
print(f"{words_cosine_similarity('добро', 'зло', w2v_embeddings):.4f}")

1.0000
0.0278
-0.1324
0.1140
-0.0324
-0.3617


Ну тут уже получили, что-то более менее осмысленное. Например, векторы для добра и зла направлены в разные стороны.

## Использование предобученных эмбедингов

Что ж, теперь мы можем смело использовать предобученные эмбединги для улучшения решения исходной задачи!

**Задача 3 (1 балл)**. Так сделайте же это! Модифицируйте модель из первой части для того, чтобы она использовала эмбединги, полученные во второй части. Обратите внимание на [полезную функцию](https://pytorch.org/docs/master/generated/torch.nn.Embedding.html#torch.nn.Embedding.from_pretrained)

In [ ]:
class UpgradedRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, 
                 tagset_size, pretrained_wieghts):
        super(UpgradedRNN, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding.from_pretrained(pretrained_wieghts, freeze=False)
        self.lstm = nn.RNN(embedding_dim, pretrained_wieghts.shape[1])

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

    def embeddings(self):
      return self.word_embeddings.weight.data.numpy()

In [ ]:
model_pretr = UpgradedRNN(32, 32, len(vocabulary), len(tag2index), 
                          model_word2vec.word_embeddings.weight)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_pretr.parameters(), lr=0.1)

train_rnn(model_pretr, loss, optimizer, data_train, data_dev, num_epoch=num_epoch)

Epoch:  0
Train:



Train loss:  30373.11995862355
Validation:



Validation loss:  3888.6857475677825
Epoch:  1
Train:



Train loss:  17498.793338479732
Validation:



Validation loss:  2878.0329634749214
Epoch:  2
Train:


...


Validation loss:  2377.953933651188
Epoch:  3
Train:



Train loss:  9931.773336561302
Validation:



Validation loss:  2121.774670802348
Epoch:  4
Train:



Train loss:  8538.239748175689
Validation:



Validation loss:  1961.8900473070353


In [ ]:
up_embeddings = model_pretr.embeddings()

print(f"{words_cosine_similarity('папа', 'папа', up_embeddings):.4f}")
print(f"{words_cosine_similarity('кот', 'собака', up_embeddings):.4f}")
print(f"{words_cosine_similarity('папа', 'мама', up_embeddings):.4f}")
print(f"{words_cosine_similarity('Алжир', 'снег', up_embeddings):.4f}")
print(f"{words_cosine_similarity('женщина', 'мужчина', up_embeddings):.4f}")
print(f"{words_cosine_similarity('добро', 'зло', up_embeddings):.4f}")

1.0000
0.0387
-0.1499
0.1142
-0.0548
-0.3495
